In [ ]:
from wordle_env import WordleEnv
from stable_baselines3 import PPO
from gymnasium.wrappers import RecordEpisodeStatistics

# calendar: dict[date -> answer]; load from CSV once and reuse
env = RecordEpisodeStatistics(
    WordleEnv(calendar=None, eval_mode=False),  # training
)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=2_500_000)

c:\Users\darcy\Documents\wordrl\wordle_env.py:67: RuntimeWarning: divide by zero encountered in log2
  self._potential = -np.log2(len(self._candidate_set))


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6        |
|    ep_rew_mean     | 11.1     |
| time/              |          |
|    fps             | 329      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 6          |
|    ep_rew_mean          | 11.4       |
| time/                   |            |
|    fps                  | 266        |
|    iterations           | 2          |
|    time_elapsed         | 15         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.03862669 |
|    clip_fraction        | 0.604      |
|    clip_range           | 0.2        |
|    entropy_loss         | -9.27

In [ ]:
eval_env = WordleEnv(target='below', eval_mode=True)
obs, _ = eval_env.reset()
done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = eval_env.step(action)

In [ ]:
print(f"Result: {reward:+}  Guessed: {info['guess']}  Answer: {info['answer']}")

Result: -1  Guessed: mohua  Answer: below


In [ ]:
eval_env.render()

'⬜⬜⬜⬜⬜\n⬜🟨🟨🟨⬜\n🟩🟨⬜⬜🟨\n🟨⬜🟨⬜⬜\n⬜🟩⬜⬜⬜\n⬜🟨⬜⬜⬜'